# Spaceship. Part 5.
## Hyperparameters tuning 

We'll load our prepared in Part 4 data, as well as the scores DataFrame, and set random seed:

In [70]:
# Random seed for reproducibility
SEED = 123

import pandas as pd

train = pd.read_csv('04_train_prepared.csv', index_col=0)
test =  pd.read_csv('04_test_prepared.csv', index_col=0)
scores_df = pd.read_csv('04_scores_df.csv', index_col=0)
test_Ids = pd.read_csv('test_Ids.csv', index_col=0).reset_index(drop=True)

train['Transported'] = [1 if i else 0 for i in train['Transported']]

Next, we'll create train_evaluate function, that will return average cross-validation ROC AUC score for a given set of parameters. 

We'll also define our classifier inside this function.  We'll put n_estimators to 90 for speed. Greater numbers may increase scores. For the scores table and submissions we'll use 500 estimators.

In [71]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

def train_evaluate(params):
    '''
    This function takes  parameters for a classifier.
    
    It returns average cross-validated ROC AUC score.
    '''
    
    # Prepare our best estimator for training
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(random_state=SEED,
                               n_estimators= 90,
                               n_jobs=-1
                               )


    # Set parameters for the model
    model.set_params(**params)
    
    # Create a StratifiedKFold object (6 splits with equal proportion of positive target values)
    skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=SEED)
    
    # An empty list for collecting scores
    test_roc_auc_scores = []
    
    # Iterate through folds
    for train_index, cv_index in skf.split(train.drop('Transported', axis=1), train['Transported']):
        # Obtain training and testing folds
        cv_train, cv_test = train.iloc[train_index], train.iloc[cv_index]
        
        # Fit the model
        model.fit(cv_train.drop('Transported', axis=1), cv_train['Transported']) 
        
        # Calculate ROC AUC score and append to the scores lists
        test_pred_proba = model.predict_proba(cv_test.drop('Transported', axis=1))[:, 1]
        test_roc_auc_scores.append(roc_auc_score(cv_test['Transported'], test_pred_proba))
        
    return np.mean(test_roc_auc_scores)
        

We'll use Optuna for our tuning. For this, we'll need to create a study:

In [72]:
import optuna

study = optuna.create_study(study_name='05_RF', direction='maximize')

[I 2023-07-29 00:55:48,164] A new study created in memory with name: 05_RF


Next, we need to define objective function to optimize, which contains range of parameters for search. We'll need to redefine this function (and start a new study) in order to change the ranges of search.

In [73]:
def objective(trial):
    params = {
        # 'n_estimators': optuna.distributions.IntDistribution(100, 1000),
        # 'criterion': optuna.distributions.CategoricalDistribution(['log_loss', 'entropy']),
        'criterion': trial.suggest_categorical('criterion', ['log_loss', 'gini']),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'max_features': trial.suggest_int('max_features', 1, 16),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 3, 500),
        "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 1e-9, 1e-1, log=True),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 30),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 1e-7, 4e-1, log=True),
        'max_samples': trial.suggest_float('max_samples', 0.3, 1)
             
         }
    return train_evaluate(params)

Now we can run our study. Let's run it for 5 seconds:

In [74]:
study.optimize(objective, timeout=5, n_jobs=-1)

[I 2023-07-29 00:56:03,771] Trial 2 finished with value: 0.847681114654475 and parameters: {'criterion': 'gini', 'max_depth': 3, 'max_features': 12, 'max_leaf_nodes': 304, 'min_impurity_decrease': 1.0491468178782143e-09, 'min_samples_leaf': 22, 'ccp_alpha': 0.0003442836526355932, 'max_samples': 0.5448877830299387}. Best is trial 2 with value: 0.847681114654475.
[I 2023-07-29 00:56:03,838] Trial 10 finished with value: 0.765388439289846 and parameters: {'criterion': 'log_loss', 'max_depth': 17, 'max_features': 10, 'max_leaf_nodes': 130, 'min_impurity_decrease': 0.05636725167979558, 'min_samples_leaf': 11, 'ccp_alpha': 0.02659517578833693, 'max_samples': 0.772917543553331}. Best is trial 2 with value: 0.847681114654475.
[I 2023-07-29 00:56:03,845] Trial 3 finished with value: 0.8593340609973499 and parameters: {'criterion': 'gini', 'max_depth': 38, 'max_features': 12, 'max_leaf_nodes': 34, 'min_impurity_decrease': 1.2416606848013118e-08, 'min_samples_leaf': 26, 'ccp_alpha': 0.00545048043

Here is what we've got in 5 seconds:

In [75]:
print("Best trial:", study.best_trial.number)
print("Best average cross-validation ROC AUC:", study.best_trial.value)
print("Best hyperparameters:", study.best_params)

Best trial: 11
Best average cross-validation ROC AUC: 0.8822049875726843
Best hyperparameters: {'criterion': 'gini', 'max_depth': 48, 'max_features': 8, 'max_leaf_nodes': 433, 'min_impurity_decrease': 5.23655263053275e-08, 'min_samples_leaf': 2, 'ccp_alpha': 1.1113424607141138e-07, 'max_samples': 0.686348320267558}


Let's take a look at the optimization history. Note, that plot is interactive, so we can zoom in:

In [76]:
# Plotting Optimization History
import optuna.visualization as vis

optimization_history_plot = vis.plot_optimization_history(study, error_bar=True)
optimization_history_plot.show()

Let's run for 5 seconds more, look at the results and at the plot:

In [77]:
study.optimize(objective, timeout=5, n_jobs=-1)

print("Best trial:", study.best_trial.number)
print("Best average cross-validation ROC AUC:", study.best_trial.value)
print("Best hyperparameters:", study.best_params)

# Plotting Optimization History
optimization_history_plot = vis.plot_optimization_history(study, error_bar=True)
optimization_history_plot.show()

[I 2023-07-29 00:56:19,912] Trial 20 finished with value: 0.8811389997869822 and parameters: {'criterion': 'gini', 'max_depth': 49, 'max_features': 6, 'max_leaf_nodes': 438, 'min_impurity_decrease': 1.1903990478055807e-06, 'min_samples_leaf': 17, 'ccp_alpha': 2.997822178147827e-06, 'max_samples': 0.913493819181181}. Best is trial 11 with value: 0.8822049875726843.
[I 2023-07-29 00:56:19,917] Trial 13 finished with value: 0.8798713747976977 and parameters: {'criterion': 'gini', 'max_depth': 48, 'max_features': 6, 'max_leaf_nodes': 496, 'min_impurity_decrease': 1.2613112975222095e-06, 'min_samples_leaf': 15, 'ccp_alpha': 4.903037219211565e-06, 'max_samples': 0.4478650712430027}. Best is trial 11 with value: 0.8822049875726843.
[I 2023-07-29 00:56:20,025] Trial 15 finished with value: 0.879253597786921 and parameters: {'criterion': 'gini', 'max_depth': 50, 'max_features': 6, 'max_leaf_nodes': 495, 'min_impurity_decrease': 2.150436648572292e-06, 'min_samples_leaf': 30, 'ccp_alpha': 3.35784

Best trial: 11
Best average cross-validation ROC AUC: 0.8822049875726843
Best hyperparameters: {'criterion': 'gini', 'max_depth': 48, 'max_features': 8, 'max_leaf_nodes': 433, 'min_impurity_decrease': 5.23655263053275e-08, 'min_samples_leaf': 2, 'ccp_alpha': 1.1113424607141138e-07, 'max_samples': 0.686348320267558}


Note, that we didn't start over, but continued previous search. We can also look at the importances of each hyperparameter:

In [78]:
# Plotting Parameter Importance
param_importance_plot = vis.plot_param_importances(study)
param_importance_plot.show()

We can plot Contour Plots of Optimization, to highlight the relationship between two hyperparameters and the trial value. Let's look at such plot:

In [79]:
# Plotting a Contour Plot in Optuna
contour_plot = vis.plot_contour(study, params=["min_impurity_decrease", "max_leaf_nodes"])
contour_plot.show()

Finally, we can save study to a file and load it later!

In [80]:
import joblib

joblib.dump(study, "05_RF.pkl")

study = joblib.load("05_RF.pkl")
    
print("Best trial:", study.best_trial.number)
print("Best average cross-validation ROC AUC:", study.best_trial.value)
print("Best hyperparameters:", study.best_params)

Best trial: 11
Best average cross-validation ROC AUC: 0.8822049875726843
Best hyperparameters: {'criterion': 'gini', 'max_depth': 48, 'max_features': 8, 'max_leaf_nodes': 433, 'min_impurity_decrease': 5.23655263053275e-08, 'min_samples_leaf': 2, 'ccp_alpha': 1.1113424607141138e-07, 'max_samples': 0.686348320267558}


Let's put our new scores in our score table. For this, we'll reuse get_scores_cv fuction from Part 4, but now we'll use the best parameters from our Optuna study:

In [81]:
%%time

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Random seed for reproducibility
SEED = 123

# Prepare our best model for training
from sklearn.ensemble import RandomForestClassifier
model_for_tests = RandomForestClassifier(random_state=SEED,
                               n_estimators= 90,
                               n_jobs=-1,
                               **study.best_params
                               )


def get_cv_scores(train, test, model, scores_df, comment = "", verbose=False, prepare_submission=False):
    
    '''
    This function takes train and test sets, as well as a model for cross validation and a DataFrame with previous scores.
    It also takes an optional comment string to comment changes.
    
    Setting verbose to True makes function printing out updated scores.

    
    It returns:
        
        -) Updated DataFrame with new:
            1) Average training ROC AUC score.
            2) Average cross-validation ROC AUC score.
            3) Average training accuracy score. 
            4) Average cross-validation accuracy score.
        
        -) A dataset for a new submission, if prepare_submission is True
    '''
    
    # Create a StratifiedKFold object (6 splits with equal proportion of positive target values)
    skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=SEED)
    
    # Empty lists for collecting scores
    train_roc_auc_scores = []
    cv_roc_auc_scores = []
    train_accuracy_scores = []
    cv_accuracy_scores = []
    
    # Iterate through folds
    for train_index, cv_index in skf.split(train.drop('Transported', axis=1), train['Transported']):
        # Obtain training and testing folds
        cv_train, cv_test = train.iloc[train_index], train.iloc[cv_index]
        
        # Fit the model
        model.fit(cv_train.drop('Transported', axis=1), cv_train['Transported']) 
        
        # Calculate scores and append to the scores lists
        train_pred_proba = model.predict_proba(cv_train.drop('Transported', axis=1))[:, 1]
        train_roc_auc_scores.append(roc_auc_score(cv_train['Transported'], train_pred_proba))
        cv_pred_proba = model.predict_proba(cv_test.drop('Transported', axis=1))[:, 1]
        cv_roc_auc_scores.append(roc_auc_score(cv_test['Transported'], cv_pred_proba))
        train_accuracy_scores.append(model.score(cv_train.drop('Transported', axis=1), cv_train['Transported']))
        cv_accuracy_scores.append(model.score(cv_test.drop('Transported', axis=1), cv_test['Transported']))
        

    # Update the scores DataFrame with average scores:
    
    scores_df.loc[len(scores_df)] = [comment, np.mean(train_roc_auc_scores), np.mean(cv_roc_auc_scores), \
                                     np.mean(train_accuracy_scores), np.mean(cv_accuracy_scores), np.nan]
    #scores_df.index = scores_df.index + 1
    #scores_df.sort_index()
    
    # Print the updated scores DataFrame
    if verbose:
        print(scores_df)
        
    submission = "prepare_submission=False"
        
    if prepare_submission:
    
        # Prepare the submission DataFrame
        test_pred = model.predict(test)
        test_pred = ["True" if i == 1 else "False" for i in test_pred]
        test_pred = pd.DataFrame(test_pred, columns=['Transported'])
        submission = pd.concat([test_Ids, test_pred], axis=1)

    
    return submission

print(model_for_tests)

get_cv_scores(train, test, model_for_tests, scores_df, comment= "optuna_test", prepare_submission=False)

scores_df

RandomForestClassifier(ccp_alpha=1.1113424607141138e-07, max_depth=48,
                       max_features=8, max_leaf_nodes=433,
                       max_samples=0.686348320267558,
                       min_impurity_decrease=5.23655263053275e-08,
                       min_samples_leaf=2, n_estimators=90, n_jobs=-1,
                       random_state=123)
CPU times: total: 2.11 s
Wall time: 2.37 s


,Changes:,Train ROC AUC,Cross-val ROC AUC,Train Accuracy,Cross-val Accuracy,Test accuracy
0,Unprocessed numeric features,0.891046,0.847367,0.830047,0.790751,0.80056
1,GroupSize,0.899330,0.854434,0.828759,0.792477,NaN
2,FamilySize,0.900522,0.854035,0.828989,0.791787,NaN
3,- GroupSize,0.896124,0.851555,0.829334,0.792938,NaN
4,1 + Deck_enc,0.927033,0.877194,0.831174,0.791672,0.79682
5,+ HomePlanet,0.930021,0.880737,0.834349,0.794893,NaN
6,+ Destination,0.930903,0.882469,0.835431,0.796503,NaN
7,+ CryoSleep,0.931807,0.882508,0.838951,0.799264,NaN
8,+ VIP,0.931491,0.882599,0.838790,0.799724,NaN
9,+ Side,0.934370,0.886205,0.844196,0.799379,NaN


Here we described the whole process. Now we'll save our study and from now on we'll be rerunning ['05_hyperparameters_2.ipynb'](05_hyperparameters_2.ipynb), loading and saving our study to a file.

In [82]:
total_seconds = pd.DataFrame([10])

joblib.dump(study, "05_RF.pkl")
scores_df.to_csv('05_scores_df.csv')
total_seconds.to_csv('05_total_seconds.csv')